# Getting Started

This notebook walks through the different types of memory you can use with the `ConversationChain`.

## ConversationBufferMemory (default)
By default, the `ConversationChain` uses `ConversationBufferMemory`: a simple type of memory that remembers all previous inputs/outputs and adds them to the context that is passed. Let's take a look at using this chain (setting `verbose=True` so we can see the prompt).

In [1]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


llm = OpenAI(temperature=0)
conversation = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory=ConversationBufferMemory()
)

ImportError: cannot import name 'BasePromptTemplate' from partially initialized module 'langchain' (most likely due to a circular import) (/Users/harrisonchase/workplace/langchain/langchain/__init__.py)

In [ ]:
conversation.predict(input="Hi there!")

In [ ]:
conversation.predict(input="I'm doing well! Just having a conversation with an AI.")

In [ ]:
conversation.predict(input="Tell me about yourself.")

## ConversationSummaryMemory
Now let's take a look at using a slightly more complex type of memory - `ConversationSummaryMemory`. This type of memory creates a summary of the conversation over time. This can be useful for condensing information from the conversation over time.

Let's walk through an example, again setting `verbose=True` so we can see the prompt.

In [ ]:
from langchain.memory import ConversationSummaryMemory

In [ ]:
conversation_with_summary = ConversationChain(
    llm=llm, 
    memory=ConversationSummaryMemory(llm=OpenAI()),
    verbose=True
)
conversation_with_summary.predict(input="Hi, what's up?")

In [ ]:
conversation_with_summary.predict(input="Tell me more about it!")

In [ ]:
conversation_with_summary.predict(input="Very cool -- what is the scope of the project?")

## ConversationBufferWindowMemory

`ConversationBufferWindowMemory` keeps a list of the interactions of the conversation over time. It only uses the last K interactions. This can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large

Let's walk through an example, again setting `verbose=True` so we can see the prompt.

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

In [ ]:
conversation_with_summary = ConversationChain(
    llm=llm, 
    # We set a low k=2, to only keep the last 2 interactions in memory
    memory=ConversationBufferWindowMemory(k=2), 
    verbose=True
)
conversation_with_summary.predict(input="Hi, what's up?")

In [ ]:
conversation_with_summary.predict(input="What's their issues?")

In [ ]:
conversation_with_summary.predict(input="Is it going well?")

In [ ]:
# Notice here that the first interaction does not appear.
conversation_with_summary.predict(input="What's the solution?")

## ConversationSummaryBufferMemory

`ConversationSummaryBufferMemory` combines the last two ideas. It keeps a buffer of recent interactions in memory, but rather than just completely flushing old interactions it compiles them into a summary and uses both. Unlike the previous implementation though, it uses token length rather than number of interactions to determine when to flush interactions.

Let's walk through an example, again setting `verbose=True` so we can see the prompt.

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory

In [ ]:
conversation_with_summary = ConversationChain(
    llm=llm, 
    # We set a very low max_token_limit for the purposes of testing.
    memory=ConversationSummaryBufferMemory(llm=OpenAI(), max_token_limit=40),
    verbose=True
)
conversation_with_summary.predict(input="Hi, what's up?")

In [ ]:
conversation_with_summary.predict(input="Just working on writing some documentation!")

In [ ]:
# We can see here that there is a summary of the conversation and then some previous interactions
conversation_with_summary.predict(input="For LangChain! Have you heard of it?")

In [ ]:
# We can see here that the summary and the buffer are updated
conversation_with_summary.predict(input="Haha nope, although a lot of people confuse it for that")

## Conversation Knowledge Graph Memory

This type of memory uses a knowledge graph to recreate memory.

In [ ]:
from langchain.memory import ConversationKGMemory

In [ ]:
llm = OpenAI(temperature=0)
from langchain.prompts.prompt import PromptTemplate

template = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human: {input}
AI:"""
prompt = PromptTemplate(
    input_variables=["history", "input"], template=template
)
conversation_with_kg = ConversationChain(
    llm=llm, 
    verbose=True, 
    prompt=prompt,
    memory=ConversationKGMemory(llm=llm)
)

In [ ]:
conversation_with_kg.predict(input="Hi, what's up?")

In [ ]:
conversation_with_kg.predict(input="My name is James and I'm helping Will. He's an engineer.")

In [ ]:
conversation_with_kg.predict(input="What do you know about Will?")